In [1]:
from spacerocks.spacerock import SpaceRock, RockCollection
from spacerocks.observing import Observatory, DetectionCatalog
from spacerocks.spice import SpiceKernel
from spacerocks.time import Time
from spacerocks.nbody import Simulation, Integrator

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np

kernel = SpiceKernel()
kernel.load("/Users/kjnapier/data/spice/latest_leapseconds.tls")
kernel.load("/Users/kjnapier/data/spice/de440s.bsp")

In [2]:
ARCSEC_PER_RAD = 206264.80624709636

In [3]:
epoch = Time.now()
planets_names = ["sun", "mercury barycenter"]
planets = [SpaceRock.from_spice(name, epoch, frame="ECLIPJ2000", origin="ssb") for name in planets_names]

In [4]:
sim = Simulation()

sim.set_epoch(epoch)
sim.set_frame("ECLIPJ2000")
sim.set_origin("ssb")

for planet in planets:
    sim.add(planet)
    
sim.move_to_center_of_mass()

In [5]:
mercury = sim.perturbers[1]
mercury.calculate_orbit()

evec_0 = np.array(mercury.evec) / np.linalg.norm(mercury.evec)

In [6]:
n_centuries = 10
sim.integrate(epoch + 100 * n_centuries * 365.25)

In [7]:
mercury = sim.perturbers[1]
mercury.calculate_orbit()

evec_1 = np.array(mercury.evec) / np.linalg.norm(mercury.evec)

In [8]:
precession_rate = np.arccos(np.dot(evec_0, evec_1)) * ARCSEC_PER_RAD / n_centuries

print(f"Mercury's pericenter precession rate due to GR is {precession_rate:.2f} arcsec/century.")

Mercury's pericenter precession rate due to GR is 42.98 arcsec/century.
